In [27]:
import pandas as pd
import numpy as np

In [28]:
data = pd.read_csv("data500K_10c.csv", header=None)


In [29]:
df = data[:25]
df.to_csv("test.csv", index = False)

In [30]:
import datetime

from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.classification import DecisionTreeClassifier
import pandas as pd
import re
import warnings
import os
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
import datetime
import numpy as np
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel



import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn.svm import SVC     ### SVM for classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import SVC
import math
from datetime import datetime

In [31]:
conf = SparkConf()
conf.setMaster('local')
conf.setAppName('spark-basic')
# sc = SparkContext(conf=conf)
sc = SparkContext.getOrCreate(conf=conf)


df= sc.textFile("test.csv").map(lambda line: line.split(","))
                                        #label , features
dataset  = df.map(lambda x: LabeledPoint(x[0], x[1:]))

(trainingData, testData) = dataset.randomSplit([0.7, 0.3])


print (dataset.collect()[0])

(0.0,[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0,69.0,70.0,71.0,72.0,73.0,74.0,75.0,76.0,77.0,78.0,79.0,80.0,81.0,82.0,83.0,84.0,85.0,86.0,87.0,88.0,89.0,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0])


In [74]:
test = pd.read_csv("test.csv")
a = test.iloc[:,[0]]
b = pd.unique(a.values.ravel())
print (b)

[9 7 8 0 4 2 6 5 3 1]


In [32]:
start = datetime.now()
model = DecisionTree.trainClassifier(trainingData,  numClasses=20, categoricalFeaturesInfo={}, impurity='gini', maxDepth=8, maxBins=32)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))    
# print(predictions.collect())

end = datetime.now() -start

print ("Decision Tree -----------------\n")
print ('Runtime : ', end)
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)

#    print(labelsAndPredictions.collect()[1])

acc =0
acc = (labelsAndPredictions.filter( lambda lp: lp[0] == lp[1]).count()) / float(testData.count())
print('Test Accurancy = ' + str(acc))

23/04/17 12:19:03 WARN DecisionTreeMetadata: DecisionTree reducing maxBins from 32 to 16 (= number of training instances)


Decision Tree -----------------

Runtime :  0:00:00.605511
Test Accurancy = 0.2


In [35]:
start = datetime.now()

model = NaiveBayes.train(trainingData, 1.0)
end = datetime.now() -start
print ("\nNaive Bayes -----------------\n")
print ('Runtime : ', end)
# Make prediction and test accuracy.
predictionAndLabel = testData.map(lambda p: (model.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda pl: pl[0] == pl[1]).count() / testData.count()
print('Test Accuracy {}'.format(accuracy))




Naive Bayes -----------------

Runtime :  0:00:00.662869
Test Accuracy 0.6


In [26]:
start = datetime.now()

model = SVMWithSGD.train(trainingData, iterations=10)
end = datetime.now() -start

print ("\nSVM -----------------\n")
print ('Runtime : ', end)
# Evaluating the model on training data
labelsAndPreds = testData.map(lambda p: (p.label, model.predict(p.features)))
acc = labelsAndPreds.filter(lambda lp: lp[0] == lp[1]).count() / float(testData.count())
print("Test Accurancy = " + str(acc))    

23/04/17 12:18:55 ERROR DataValidators: Classification labels should be 0 or 1. Found 17 invalid labels


Py4JJavaError: An error occurred while calling o314.trainSVMModelWithSGD.
: org.apache.spark.SparkException: Input validation failed.
	at org.apache.spark.mllib.regression.GeneralizedLinearAlgorithm.run(GeneralizedLinearAlgorithm.scala:252)
	at org.apache.spark.mllib.api.python.PythonMLLibAPI.trainRegressionModel(PythonMLLibAPI.scala:92)
	at org.apache.spark.mllib.api.python.PythonMLLibAPI.trainSVMModelWithSGD(PythonMLLibAPI.scala:251)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:76)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:52)
	at java.base/java.lang.reflect.Method.invoke(Method.java:577)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.Abstract

In [34]:
start = datetime.now()

model = LogisticRegressionWithLBFGS.train(trainingData, iterations=10,numClasses=)
end = datetime.now() -start
print ("\nLogisticRegression -----------------\n")
print ('Runtime : ', end)
# Evaluating the model on training data
labelsAndPreds = testData.map(lambda p: (p.label, model.predict(p.features)))
acc = labelsAndPreds.filter(lambda lp: lp[0] == lp[1]).count() / float(testData.count())
print("Test Accurancy = " + str(acc))

23/04/17 12:26:36 WARN Instrumentation: [13c30597] Initial coefficients will be ignored! Its dimensions (1, 100) did not match the expected size (10, 100)
23/04/17 12:26:36 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/04/17 12:26:36 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


Py4JJavaError: An error occurred while calling o459.trainLogisticRegressionModelWithLBFGS.
: org.apache.spark.SparkException: Multinomial models contain a matrix of coefficients, use coefficientMatrix instead.
	at org.apache.spark.ml.classification.LogisticRegressionModel.coefficients(LogisticRegression.scala:1085)
	at org.apache.spark.mllib.classification.LogisticRegressionWithLBFGS.runWithMlLogisticRegression$1(LogisticRegression.scala:344)
	at org.apache.spark.mllib.classification.LogisticRegressionWithLBFGS.run(LogisticRegression.scala:348)
	at org.apache.spark.mllib.classification.LogisticRegressionWithLBFGS.run(LogisticRegression.scala:316)
	at org.apache.spark.mllib.classification.LogisticRegressionWithLBFGS.run(LogisticRegression.scala:246)
	at org.apache.spark.mllib.api.python.PythonMLLibAPI.trainRegressionModel(PythonMLLibAPI.scala:92)
	at org.apache.spark.mllib.api.python.PythonMLLibAPI.trainLogisticRegressionModelWithLBFGS(PythonMLLibAPI.scala:311)
	at java.base/jdk.interna